<a href="https://colab.research.google.com/github/lblum95/AML/blob/master/task3/1DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Mounted at /content/drive


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.metrics import Precision, Recall
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import numpy as np
from sklearn.decomposition import KernelPCA, PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import StackingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2, f_classif, mutual_info_classif, \
    SelectFromModel
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,MaxPooling1D,Flatten,BatchNormalization

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
x_train = pd.read_csv("data/X_train_features.csv", index_col=0, header=0, low_memory=False)
y_train = pd.read_csv("data/y_train.csv", index_col=0, header=0)
x_test = pd.read_csv("data/X_test_features.csv", index_col=0, header=0, low_memory=False)
#Only take time avg. segment
normal_features=x_train.iloc[:,57:]
#normal_features=x_train
print(normal_features.shape)
normal_features=normal_features[y_train['y']<3]
print(normal_features.shape)
imputer = SimpleImputer()

scaler = StandardScaler()
feature_sel = SelectFromModel(RandomForestClassifier())
pca = KernelPCA(kernel='rbf', random_state=42)
features=imputer.fit_transform(normal_features)
selected=scaler.fit_transform(features)
y_tr=y_train[y_train['y']<3]
#selected=feature_sel.fit_transform(selected,y_tr)
#selected=pca.fit_transform(selected)

(5117, 1820)
(4947, 1820)


In [ ]:
# different loss for Keras to optimize, performed worse than categorical_crossentropy
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    #f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    #f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
    
def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Implementation of Focal Loss from the paper in multiclass classification
    Formula:
        loss = -alpha*((1-p)^gamma)*log(p)
    Parameters:
        alpha -- the same as wighting factor in balanced cross entropy
        gamma -- focusing parameter for modulating factor (1-p)
    Default value:
        gamma -- 2.0 as mentioned in the paper
        alpha -- 0.25 as mentioned in the paper
    """
    def focal_loss(y_true, y_pred):
        # Define epsilon so that the backpropagation will not result in NaN
        # for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        #y_pred = y_pred + epsilon
        # Clip the prediction value
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        # Calculate cross entropy
        cross_entropy = -y_true*K.log(y_pred)
        # Calculate weight that consists of  modulating factor and weighting factor
        weight = alpha * y_true * K.pow((1-y_pred), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.sum(loss, axis=1)
        return loss
    
    return focal_loss

In [ ]:
input_dim=selected.shape[1]
print(input_dim)
y_tr=y_train[y_train['y']<3]
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_tr)

1820


OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

In [ ]:
pd.set_option('display.max_columns', None)
x_train.head()
x_train

R_ampl_mean  R_ampl_median  R_ampl_perc5  R_ampl_perc95    R_ampl_sd  \
0  1186.293185    1231.580081    793.458171    1488.904935   243.022609   
1   317.920824     324.005372    248.446898     446.807099    93.097314   
2   647.445417     642.303309    525.281618     767.867806    97.833123   
3   776.715649     887.776256    333.101396     951.305024   231.155884   
4   997.732477     544.058496    258.669653    3461.737872  1303.418298   

   R_nr_peaks   len_mean  len_median  len_perc5  len_perc95  len_sd  \
0        13.0  11.000000   11.000000  11.000000   11.000000     0.0   
1        30.0  28.896667   28.896667  28.896667   28.896667     0.0   
2        39.0  27.983333   27.983333  27.983333   27.983333     0.0   
3        31.0  28.426667   28.426667  28.426667   28.426667     0.0   
4        12.0  11.756667   11.756667  11.756667   11.756667     0.0   

   Qual_mean  Qual_median  Qual_perc5  Qual_perc95   Qual_sd  Q_ampl_mean  \
0   0.748455     0.843203    0.038058     1.001202  0.278681  -195.063334   
1   0.747439     0.781426    0.309135     0.996052  0.208130   -23.828665   
2   0.704328     0.763004    0.073176     0.995712  0.252187  -108.946172   
3   0.614607     0.641604    0.189825     0.997587  0.249708   -56.932654   
4   0.781622     0.897203   -0.010364     1.010614  0.298740  -383.153624   

   Q_ampl_median  Q_ampl_perc5  Q_ampl_perc95    Q_ampl_sd  Q_nr_peaks  \
0    -187.285993   -414.939487      53.541782   149.818680        12.0   
1     -13.719774    -75.716209      12.890946    39.872505        27.0   
2    -103.960968   -303.169773      -4.976926    79.937980        36.0   
3     -57.077454    -95.114664     -19.865740    25.593990        29.0   
4     -41.772524  -2037.857993      87.304486  1025.355002         9.0   

   Q_diff_mean  Q_diff_median  Q_diff_perc5  Q_diff_perc95  Q_diff_sd  \
0     0.871515       0.866667      0.438333       1.366667   0.344167   
1     1.011026       0.986667      0.790000       1.526667   0.256821   
2     0.764667       0.713333      0.639000       1.326333   0.198186   
3     0.922619       0.885000      0.811167       0.962167   0.186978   
4     1.371667       1.066667      0.772667       2.656833   0.740143   

   S_ampl_mean  S_ampl_median  S_ampl_perc5  S_ampl_perc95    S_ampl_sd  \
0  -251.327443    -216.931724   -502.120301     -70.714117   139.466348   
1  -212.560382    -207.339120   -262.106450    -171.454539    34.405823   
2  -107.788493    -113.294065   -258.254429      41.360099   109.544605   
3  -370.279282    -188.803700  -1171.378006    -151.768960   372.685425   
4  -340.629655     -84.118320  -2074.184608     402.642821  1083.530324   

   S_nr_peaks  S_diff_mean  S_diff_median  S_diff_perc5  S_diff_perc95  \
0        13.0     0.799444       0.855000      0.465500       0.933000   
1        30.0     0.938391       0.980000      0.688000       1.062000   
2        39.0     0.706404       0.706667      0.680000       0.737167   
3        31.0     0.886556       0.890000      0.759667       1.026833   
4        11.0     1.086667       0.988333      0.487333       1.891000   

   S_diff_sd  QRS_diff_mean  QRS_diff_median  QRS_diff_perc5  QRS_diff_perc95  \
0   0.162929       0.132222         0.123333        0.102500         0.186667   
1   0.139645       0.097037         0.090000        0.077667         0.105667   
2   0.025745       0.108796         0.101667        0.066667         0.162500   
3   0.090445       0.126092         0.130000        0.088000         0.160667   
4   0.481668       0.232917         0.208333        0.151167         0.351000   

   QRS_diff_sd   HRV_RMSSD   HRV_MeanNN    HRV_SDNN    HRV_SDSD  HRV_CVNN  \
0     0.031486  201.052785   801.388889  168.980654  210.831447  0.210860   
1     0.046714  152.100374   938.505747  142.782958  154.877728  0.152139   
2     0.037999   13.445474   706.578947   15.528465   13.626041  0.021977   
3     0.023782   96.098210   889.777778   61.631294   97.748182  0.069266   
4     0.07

In [ ]:
stacked=[]
length=180
for row in range(len(selected)):
  current_row=[]
  for time in range(length):
    current_signal=[]
    for feature in range(5):
      current_signal.append(selected[row,57+feature*length+time])
    current_row.append(current_signal)
  stacked.append(current_row)

In [ ]:
print(len(stacked))
print(len(stacked[0]))
print(len(stacked[0][0]))
print(stacked[0])
print(stacked[1])
print(stacked[2])

5117
180
5
[[-27.17986000319896, -11.270503071907074, -88.4518000450976, 0.5477011895657191, 35.77194661917703], [-24.48126469977149, -13.40243062987536, -83.69499246141869, 2.933849549148073, 34.47874981356154], [-21.13987209613093, -14.964523795885821, -76.65521940576282, 6.135881651784048, 32.80063578468254], [-17.302759955108787, -9.426112990775787, -68.03868361053847, 9.131318987721604, 31.087150624727148], [-13.140747892734034, -5.0199223195209965, -58.68624182045335, 12.024705878318615, 29.66976324315312], [-8.786824625492072, -1.5980525547511313, -53.14032216405328, 14.934697784494611, 28.669840321929534], [-4.2760337114858284, 0.060918109510124725, -48.87160481190935, 18.337842934623815, 27.8490371521808], [0.4858396472753161, 2.5959201096980222, -43.359642418996465, 24.066204231202025, 26.620188289969917], [5.7048448061517885, 6.577252897488399, -35.24513043483207, 29.35220321353103, 24.247425854773702], [11.630769276416734, 12.465453069666575, -23.11981699825673, 33.84656896

In [ ]:
stacked=np.array(stacked)

In [ ]:
# adjusted those params
input_dim=180
dropout=0.2
def get_model():
	# create model
  model = Sequential()
  # Normalization on and off tried
  model.add(BatchNormalization())
  model.add(Dropout(dropout))
  model.add(Dense(input_dim, activation='selu'))
  model.add(Dropout(dropout))
  model.add(Dense(input_dim, activation='selu'))
  model.add(Dropout(dropout))
  model.add(Dense(input_dim, activation='selu'))
  model.add(Dropout(dropout))
  model.add(Dense(3, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[Precision(), Recall()])
  return model

In [ ]:
skf=StratifiedKFold(n_splits=5)
for train_index,test_index in skf.split(selected,y_tr):
  X_train = selected[train_index]
  X_test = selected[test_index]
  Y_train = enc.transform(y_tr.iloc[train_index]).toarray()
  Y_test = y_tr.iloc[test_index]
  model = get_model()
  early = EarlyStopping(monitor="val_loss", mode="auto", patience=20, verbose=1)
  redonplat = ReduceLROnPlateau(monitor="val_loss", mode="auto", patience=10, verbose=2)
  callbacks_list = [early, redonplat]  # early
  model.fit(X_train, Y_train, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
  model.summary()
  Y_pred=enc.inverse_transform(model.predict(X_test))
  print(confusion_matrix(Y_test, Y_pred))
  print('Score')
  print(f1_score(Y_test, Y_pred, average='micro'))
  print()
  print()
  break

Epoch 1/1000
112/112 - 1s - loss: 1.3844 - precision_95: 0.5851 - recall_95: 0.5636 - val_loss: 0.7921 - val_precision_95: 0.7049 - val_recall_95: 0.6515
Epoch 2/1000
112/112 - 1s - loss: 1.0713 - precision_95: 0.6219 - recall_95: 0.5922 - val_loss: 0.6842 - val_precision_95: 0.7127 - val_recall_95: 0.6515
Epoch 3/1000
112/112 - 0s - loss: 0.9307 - precision_95: 0.6580 - recall_95: 0.6170 - val_loss: 0.6810 - val_precision_95: 0.7352 - val_recall_95: 0.6591
Epoch 4/1000
112/112 - 1s - loss: 0.8225 - precision_95: 0.6756 - recall_95: 0.6347 - val_loss: 0.6701 - val_precision_95: 0.7345 - val_recall_95: 0.6566
Epoch 5/1000
112/112 - 0s - loss: 0.7360 - precision_95: 0.6965 - recall_95: 0.6549 - val_loss: 0.7071 - val_precision_95: 0.7095 - val_recall_95: 0.6414
Epoch 6/1000
112/112 - 0s - loss: 0.6971 - precision_95: 0.7151 - recall_95: 0.6723 - val_loss: 0.6742 - val_precision_95: 0.7258 - val_recall_95: 0.6818
Epoch 7/1000
112/112 - 0s - loss: 0.6642 - precision_95: 0.7174 - recall_95:

In [ ]:
#played with those params, filter, kernel size, and amount of dense layers at the end
input_dim=256
dropout=0.1
def get_bottle():
  model = Sequential()
  model.add(BatchNormalization())
  model.add(Conv1D(64, 3, activation='selu', input_shape=(180, 5)))
  model.add(MaxPooling1D(3))
  model.add(Dropout(dropout))
  model.add(Conv1D(64, 3, activation='selu'))
  model.add(MaxPooling1D(3))
  model.add(Flatten())
  model.add(Dropout(dropout))
  model.add(Dense(input_dim, activation='selu'))
  model.add(Dropout(dropout))
  model.add(Dense(input_dim/2, activation='selu'))
  model.add(Dropout(dropout))
  model.add(Dense(3, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[Precision(), Recall()])
  return model

skf=StratifiedKFold(n_splits=5)
for train_index,test_index in skf.split(stacked,y_tr):
  X_train = stacked[train_index]
  X_test = stacked[test_index]
  Y_train = enc.transform(y_tr.iloc[train_index]).toarray()
  Y_test = y_tr.iloc[test_index]
  model = get_bottle()
  early = EarlyStopping(monitor="val_loss", mode="auto", patience=20, verbose=1)
  redonplat = ReduceLROnPlateau(monitor="val_loss", mode="auto", patience=10, verbose=2)
  callbacks_list = [early, redonplat]  # early

  model.fit(X_train, Y_train,batch_size=len(X_train), epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
  Y_pred=enc.inverse_transform(model.predict(X_test))
  print('Score')
  print(f1_score(Y_test, Y_pred, average='micro'))
  print()
  print()
  model.summary()
  break

Epoch 1/1000
1/1 - 0s - loss: 1.6701 - precision_105: 0.2282 - recall_105: 0.1733 - val_loss: 1.2485 - val_precision_105: 0.5833 - val_recall_105: 0.5657
Epoch 2/1000
1/1 - 0s - loss: 1.2422 - precision_105: 0.5823 - recall_105: 0.5673 - val_loss: 1.1172 - val_precision_105: 0.6343 - val_recall_105: 0.6263
Epoch 3/1000
1/1 - 0s - loss: 1.2315 - precision_105: 0.6402 - recall_105: 0.6327 - val_loss: 0.9880 - val_precision_105: 0.6451 - val_recall_105: 0.6288
Epoch 4/1000
1/1 - 0s - loss: 1.1756 - precision_105: 0.6261 - recall_105: 0.6192 - val_loss: 0.8770 - val_precision_105: 0.6736 - val_recall_105: 0.6515
Epoch 5/1000
1/1 - 0s - loss: 1.0858 - precision_105: 0.6216 - recall_105: 0.6122 - val_loss: 0.8283 - val_precision_105: 0.6841 - val_recall_105: 0.6616
Epoch 6/1000
1/1 - 0s - loss: 0.9954 - precision_105: 0.6385 - recall_105: 0.6240 - val_loss: 0.8062 - val_precision_105: 0.6684 - val_recall_105: 0.6313
Epoch 7/1000
1/1 - 0s - loss: 0.9213 - precision_105: 0.6639 - recall_105: 0